<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in Toronto using Google Geocoding</a>
    
3. <a href="#item3">Explore Neighborhoods in Toronto using Lesson CSV file</a>
    
4. <a href="#item4">Analyze Each Neighborhood</a>

5. <a href="#item5">Cluster Neighborhoods</a>

6. <a href="#item6">Examine Clusters</a>    
</font>
</div>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [2]:
import pandas as pd #pandas
import requests # library to handle requests
import numpy as np #numpy
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
#api keys
google_api_key = 'key' # Google API key
CLIENT_ID = 'key' # your Foursquare ID
CLIENT_SECRET = 'key' # your Foursquare Secret
VERSION = 'date' # Foursquare API version

<a id='item1'></a>
## 1. Download and Explore Dataset

Let's use pandas built-in function to parse html and create dataframe:

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

r = requests.get(url)
df_list = pd.read_html(r.text) # this parses all the tables in webpages to a list
df = df_list[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Take a quick look on a shape of dataframe:

In [5]:
df.shape

(180, 3)

Drop rows where postal codes are not assigned to any boroughs

In [6]:
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Take a look, that all is ok

In [7]:
df.shape

(103, 3)

<a id='item2'></a>
## 2. Explore Neighborhoods in Toronto using Google Geocoding

I try to use Google Maps API to get coordinates of boroughs.
This is a function to get latitude and longitude using Google Geocoding API and
check the function with one example address:

In [69]:
def get_coordinates(api_key, address, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(api_key, address)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] # get geographical coordinates
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]
    
address = 'Red Square, Moscow, Russia'
berlin_center = get_coordinates(google_api_key, address, True)
print('Coordinate of {}: {}'.format(address, berlin_center))

Google Maps API JSON result => {'results': [{'address_components': [{'long_name': 'Red Square', 'short_name': 'Red Square', 'types': ['route']}, {'long_name': 'Tsentralnyy administrativnyy okrug', 'short_name': 'Tsentralnyy administrativnyy okrug', 'types': ['political', 'sublocality', 'sublocality_level_1']}, {'long_name': 'Moskva', 'short_name': 'Moskva', 'types': ['locality', 'political']}, {'long_name': 'Tverskoy', 'short_name': 'Tverskoy', 'types': ['administrative_area_level_3', 'political']}, {'long_name': 'Moskva', 'short_name': 'Moskva', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Russia', 'short_name': 'RU', 'types': ['country', 'political']}, {'long_name': '109012', 'short_name': '109012', 'types': ['postal_code']}], 'formatted_address': 'Red Square, Moskva, Russia, 109012', 'geometry': {'location': {'lat': 55.75393030000001, 'lng': 37.620795}, 'location_type': 'GEOMETRIC_CENTER', 'viewport': {'northeast': {'lat': 55.75527928029151, 'lng': 37.62214

Then let's get all neighborhoods coords

In [65]:
print('Obtaining location latlons: ', end='')
lat = []
lon = []
city = 'Toronto, '
for address in df['Neighborhood']:
    borough = df.loc[df['Neighborhood'] == address, 'Borough'].iloc[0]
    lat1, lon1 = get_coordinates(google_api_key, city + borough + ', ' + address)
    if lat1 is None:
        lat1 = 'NO LAT'
    if lon1 is None:
        lon1 = 'NO LON'
    lat.append(lat1)
    lon.append(lon1)
    print(' .', end='')
print(' done.')

Obtaining location latlons:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


And transform it to new dataframe

In [66]:
df_locations = pd.DataFrame({'Postal Code': df['Postal Code'],
                             'Borough': df['Borough'],
                             'Neighborhood': df['Neighborhood'],
                             'Latitude': lat,
                             'Longitude': lon,
                             })

In [67]:
df_locations.head(100)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752777,-79.326440
1,M4A,North York,Victoria Village,43.735735,-79.312418
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.638834,-79.381878
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722774,-79.450928
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662278,-79.391527
...,...,...,...,...,...
95,M1X,Scarborough,Upper Rouge,43.809571,-79.187170
96,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.665719,-79.368719
97,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648611,-79.381667
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.650939,-79.491798


Save this dataframe to a pickle

In [68]:
df_locations.to_pickle('./toronto_neighborhoods_with_latlon.pkl')   

Create a map with neighborhoods markers using Folium

In [72]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [73]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_locations['Latitude'], df_locations['Longitude'], df_locations['Borough'], df_locations['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<a id='item3'></a>
## 3. Explore Neighborhoods in Toronto using Lesson CSV file
Then, I want to compare the neighborhoods coordinates from Google with coordinates from file provided in the assignment

In [75]:
geo_from_lesson = pd.read_csv('./Geospatial_Coordinates.csv')
geo_from_lesson.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Create a sorted list of latitudes and longitudes as in initial dataframe

In [85]:
lat_fl = []
lon_fl = []
for postal in df['Postal Code']:
    lat1 = geo_from_lesson.loc[geo_from_lesson["Postal Code"]==postal]['Latitude'].iloc[0]
    lon1 = geo_from_lesson.loc[geo_from_lesson["Postal Code"]==postal]['Longitude'].iloc[0]
    if lat1 is None:
        lat1 = 'NO LAT'
    if lon1 is None:
        lon1 = 'NO LON'
    lat_fl.append(lat1)
    lon_fl.append(lon1)
print(lat_fl)
print(lon_fl)

[43.7532586, 43.725882299999995, 43.6542599, 43.718517999999996, 43.6623015, 43.6678556, 43.806686299999996, 43.745905799999996, 43.7063972, 43.6571618, 43.709577, 43.6509432, 43.7845351, 43.72589970000001, 43.695343900000005, 43.6514939, 43.6937813, 43.6435152, 43.7635726, 43.67635739999999, 43.644770799999996, 43.6890256, 43.7709921, 43.7090604, 43.6579524, 43.669542, 43.773136, 43.8037622, 43.7543283, 43.7053689, 43.65057120000001, 43.66900510000001, 43.7447342, 43.7785175, 43.7679803, 43.685347, 43.6408157, 43.647926700000006, 43.7279292, 43.7869473, 43.737473200000004, 43.6795571, 43.6471768, 43.6368472, 43.711111700000004, 43.7574902, 43.7390146, 43.6689985, 43.6481985, 43.713756200000006, 43.7563033, 43.716316, 43.789053, 43.7284964, 43.6595255, 43.7332825, 43.6911158, 43.7247659, 43.692657000000004, 43.7701199, 43.7616313, 43.7280205, 43.7116948, 43.67318529999999, 43.706876, 43.7574096, 43.752758299999996, 43.7127511, 43.6969476, 43.6616083, 43.696319, 43.750071500000004, 43.7

Then create a dataframe with lat&lon from lesson csv 

In [86]:
df_locations_fl = pd.DataFrame({'Postal Code': df['Postal Code'],
                             'Borough': df['Borough'],
                             'Neighborhood': df['Neighborhood'],
                             'Latitude': lat_fl,
                             'Longitude': lon_fl,
                             })

In [87]:
df_locations_fl.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [88]:
df_locations.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752777,-79.326440
1,M4A,North York,Victoria Village,43.735735,-79.312418
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.638834,-79.381878
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722774,-79.450928
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662278,-79.391527


We can see some differences in coordinates in both dataframes.
I create a map with neighborhoods markers using Folium from 'lesson' dataframe, so we can compare it with 'Google' dataframe

In [89]:
# create map of New York using latitude and longitude values
map_toronto_fl = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_locations_fl['Latitude'], df_locations_fl['Longitude'], df_locations_fl['Borough'], df_locations_fl['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_fl)  
    
map_toronto_fl

<a id='item4'></a>
## 4. Analyze Each Neighborhood

I decide to use 'lesson' dataframe in further evaluation

All of the following function and plotting repeat 'New York' notebook.

In [90]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [91]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [92]:
toronto_venues = getNearbyVenues(names=df_locations_fl['Neighborhood'],
                                   latitudes=df_locations_fl['Latitude'],
                                   longitudes=df_locations_fl['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [93]:
toronto_venues.shape

(2132, 7)

In [94]:
toronto_venues.to_pickle('./toronto_venues.pkl')

Here we get definitely less venues comparing to New York (2132 in all Toronto, 3132 in only Manhattan)

In [96]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
...,...,...,...,...,...,...
"Willowdale, Willowdale West",6,6,6,6,6,6
Woburn,3,3,3,3,3,3
Woodbine Heights,8,8,8,8,8,8


In [97]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 280 uniques categories.


In [98]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [99]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
91,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
92,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.0
93,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0


In [100]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Clothing Store   0.2
1                     Lounge   0.2
2             Breakfast Spot   0.2
3  Latin American Restaurant   0.2
4               Skating Rink   0.2


----Alderwood, Long Branch----
                venue  freq
0         Pizza Place  0.22
1            Pharmacy  0.11
2                 Pub  0.11
3  Athletics & Sports  0.11
4        Dance Studio  0.11


----Bathurst Manor, Wilson Heights, Downsview North----
              venue  freq
0              Bank  0.10
1       Coffee Shop  0.10
2        Restaurant  0.05
3  Sushi Restaurant  0.05
4       Supermarket  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4   Miscellaneous Shop  0.00


----Bedford Park, Lawrence Manor East----
                     venue  freq
0               Restaurant  0.08
1       Italian Restaurant  0.08
2           Sa

          venue  freq
0         Trail  0.25
1         Field  0.25
2  Tennis Court  0.25
3  Hockey Arena  0.25
4   Yoga Studio  0.00


----India Bazaar, The Beaches West----
                  venue  freq
0  Fast Food Restaurant  0.10
1        Sandwich Place  0.10
2           Pizza Place  0.05
3          Liquor Store  0.05
4            Steakhouse  0.05


----Kennedy Park, Ionview, East Birchmount Park----
               venue  freq
0     Discount Store   0.2
1  Convenience Store   0.2
2        Bus Station   0.2
3   Department Store   0.2
4        Coffee Shop   0.2


----Kensington Market, Chinatown, Grange Park----
                           venue  freq
0                           Café  0.08
1                    Coffee Shop  0.07
2  Vegetarian / Vegan Restaurant  0.05
3          Vietnamese Restaurant  0.05
4             Mexican Restaurant  0.05


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
               venue  freq
0        Pizza Place  0.25
1        

4     Mexican Restaurant  0.00


----Westmount----
                venue  freq
0         Pizza Place  0.25
1         Coffee Shop  0.12
2      Sandwich Place  0.12
3  Chinese Restaurant  0.12
4        Intersection  0.12


----Weston----
                        venue  freq
0                        Park  0.67
1           Convenience Store  0.33
2                 Yoga Studio  0.00
3                 Men's Store  0.00
4  Modern European Restaurant  0.00


----Wexford, Maryvale----
                       venue  freq
0  Middle Eastern Restaurant  0.33
1                Auto Garage  0.17
2             Breakfast Spot  0.17
3                     Bakery  0.17
4             Sandwich Place  0.17


----Willowdale, Willowdale East----
              venue  freq
0  Ramen Restaurant  0.09
1    Sandwich Place  0.06
2       Coffee Shop  0.06
3       Pizza Place  0.06
4              Café  0.06


----Willowdale, Willowdale West----
           venue  freq
0       Pharmacy  0.17
1  Grocery Store  0.17
2        

In [101]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [104]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Skating Rink,Clothing Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,"Alderwood, Long Branch",Pizza Place,Dance Studio,Gym,Pharmacy,Coffee Shop,Sandwich Place,Athletics & Sports,Pub,Women's Store,Distribution Center
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Sandwich Place,Sushi Restaurant,Ice Cream Shop,Deli / Bodega,Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Fried Chicken Joint
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Diner,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,"Bedford Park, Lawrence Manor East",Restaurant,Sandwich Place,Italian Restaurant,Coffee Shop,Pizza Place,Café,Pub,Sushi Restaurant,Liquor Store,Butcher


<a id='item5'></a>
## 5. Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [125]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:200] 

array([0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0,
       4, 0, 1, 0, 0, 4, 1, 0, 1, 0, 0, 0, 3, 0, 0, 0, 4, 1, 0, 0, 0, 1,
       0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 4, 1,
       0, 0, 4, 0, 0, 1, 2])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [138]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_locations_fl

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

print(toronto_merged.shape)
toronto_merged_drop = toronto_merged[toronto_merged['Cluster Labels'].notna()] #drop some NaN rows after k-means
print(toronto_merged_drop.shape)

(103, 16)
(99, 16)


Finally, let's visualize the resulting clusters

In [139]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_drop['Latitude'], toronto_merged_drop['Longitude'], toronto_merged_drop['Neighborhood'], toronto_merged_drop['Cluster Labels'].astype(int)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item6'></a>
## 6. Examine Clusters

Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster.
For example, cluster 4 can be Pizza Cluster.

In [141]:
toronto_merged_drop.loc[toronto_merged_drop['Cluster Labels'] == 0, toronto_merged_drop.columns[[1] + list(range(5, toronto_merged_drop.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0.0,Grocery Store,Coffee Shop,Hockey Arena,Portuguese Restaurant,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
2,Downtown Toronto,0.0,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Restaurant,Café,Theater,Historic Site,Performing Arts Venue
3,North York,0.0,Furniture / Home Store,Clothing Store,Accessories Store,Boutique,Event Space,Vietnamese Restaurant,Miscellaneous Shop,Coffee Shop,Gift Shop,Donut Shop
4,Downtown Toronto,0.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,College Auditorium
7,North York,0.0,Gym,Japanese Restaurant,Coffee Shop,Sporting Goods Shop,Restaurant,Beer Store,Asian Restaurant,Bike Shop,Italian Restaurant,Athletics & Sports
...,...,...,...,...,...,...,...,...,...,...,...,...
97,Downtown Toronto,0.0,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Gym,Seafood Restaurant,Salad Place,Steakhouse,Deli / Bodega
99,Downtown Toronto,0.0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Mediterranean Restaurant,Men's Store,Café,Pub,Yoga Studio
100,East Toronto,0.0,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Light Rail Station,Farmers Market,Comic Shop,Park,Pizza Place
101,Etobicoke,0.0,Baseball Field,Women's Store,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Dim Sum Restaurant


In [142]:
toronto_merged_drop.loc[toronto_merged_drop['Cluster Labels'] == 1, toronto_merged_drop.columns[[1] + list(range(5, toronto_merged_drop.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1.0,Food & Drink Shop,Park,Fireworks Store,Women's Store,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
21,York,1.0,Park,Pool,Women's Store,Greek Restaurant,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
35,East York,1.0,Park,Intersection,Convenience Store,Metro Station,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
61,Central Toronto,1.0,Park,Bus Line,Swim School,Fast Food Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
64,York,1.0,Park,Convenience Store,Women's Store,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
66,North York,1.0,Park,Convenience Store,Women's Store,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
83,Central Toronto,1.0,Gym,Park,Women's Store,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
85,Scarborough,1.0,Park,Playground,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
91,Downtown Toronto,1.0,Park,Trail,Playground,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
98,Etobicoke,1.0,Park,River,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop


In [143]:
toronto_merged_drop.loc[toronto_merged_drop['Cluster Labels'] == 2, toronto_merged_drop.columns[[1] + list(range(5, toronto_merged_drop.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,2.0,Cafeteria,Women's Store,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


In [144]:
toronto_merged_drop.loc[toronto_merged_drop['Cluster Labels'] == 3, toronto_merged_drop.columns[[1] + list(range(5, toronto_merged_drop.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,Etobicoke,3.0,Drugstore,Rental Car Location,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Women's Store


In [145]:
toronto_merged_drop.loc[toronto_merged_drop['Cluster Labels'] == 4, toronto_merged_drop.columns[[1] + list(range(5, toronto_merged_drop.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,4.0,Fast Food Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Field
8,East York,4.0,Pizza Place,Gym / Fitness Center,Gastropub,Intersection,Bank,Café,Athletics & Sports,Fast Food Restaurant,Pet Store,Pharmacy
10,North York,4.0,Pizza Place,Japanese Restaurant,Asian Restaurant,Sushi Restaurant,Pub,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run
50,North York,4.0,Pizza Place,Furniture / Home Store,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
63,York,4.0,Pizza Place,Caribbean Restaurant,Convenience Store,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
70,Etobicoke,4.0,Pizza Place,Sandwich Place,Intersection,Middle Eastern Restaurant,Coffee Shop,Discount Store,Chinese Restaurant,Women's Store,Dog Run,Distribution Center
72,North York,4.0,Butcher,Grocery Store,Pharmacy,Pizza Place,Bank,Coffee Shop,Donut Shop,Diner,Discount Store,Distribution Center
77,Etobicoke,4.0,Sandwich Place,Pizza Place,Bus Line,Mobile Phone Shop,Women's Store,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run
82,Scarborough,4.0,Pizza Place,Rental Car Location,Fast Food Restaurant,Fried Chicken Joint,Italian Restaurant,Thai Restaurant,Chinese Restaurant,Gas Station,Pharmacy,Noodle House
89,Etobicoke,4.0,Grocery Store,Pizza Place,Fast Food Restaurant,Pharmacy,Beer Store,Sandwich Place,Fried Chicken Joint,Dog Run,Dim Sum Restaurant,Diner


### Thank you for reading this notebook!
Best regards, Georgy